In [1]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn import metrics
import os

In [2]:
#### load data files ###

# Change file name prefix here and you are good to go:D
# prefix = "train_"
# prefix = "A_"

# Generate data file name list
file_names = ["Application.csv","Personas.csv","History_Application.csv","History_Payment.csv"]
# for idx, val in enumerate(file_names):
#     file_names[idx] = prefix + val  

# load data files
prefix = "train_"
d01 = pd.read_csv(f'./data/{prefix}{file_names[0]}')
d02 = pd.read_csv(f'./data/{prefix}{file_names[1]}')
d03 = pd.read_csv(f'./data/{prefix}{file_names[2]}')
d04 = pd.read_csv(f'./data/{prefix}{file_names[3]}')
prefix = "A_"
# load data files
d05 = pd.read_csv(f'./data/{prefix}{file_names[0]}')
d06 = pd.read_csv(f'./data/{prefix}{file_names[1]}')
d07 = pd.read_csv(f'./data/{prefix}{file_names[2]}')
d08 = pd.read_csv(f'./data/{prefix}{file_names[3]}')

# Concat
d1 = pd.concat([d01, d05], axis=0)
d2 = pd.concat([d02, d06], axis=0)
d3 = pd.concat([d03, d07], axis=0)
d4 = pd.concat([d04, d08], axis=0)

In [3]:
### Functions

# 取一个列的众数
def get_mode(df, col):
    return pd.Series(df[col]).mode()[0]

# 置多个列的空值为所属列的平均数
def fill_mean(df, cols):
    dict = {}
    for col in cols:
        dict[col] = df[col].mean()
    return df.fillna(dict)

# 置多个列的空值为所属列的众数
def fill_mode(df, cols):
    dict = {}
    for col in cols:
        dict[col] = get_mode(df, col)
    return df.fillna(dict)

# 把多个离散型的列转化为one-hot编码列
def dummy_catgorical(df, cols, keep_n_cats):
    print('initial num of cols: ' + str(len(df.columns.to_list())))
    for col in cols:
#         # 保存众数
#         mode = pd.Series(df[col]).mode()[0]
        # 离散变量太多的，低频的值归一类
        
        idx_list = df[col].value_counts().index.to_list()
        print(col+'has'+str(len(idx_list)))
        if (len(idx_list) > keep_n_cats):
            print(' col含有过多值, skipping')
#             # 把n后面的先置空,实际上归为一类，但dummy_na=1
#             for val in idx_list[keep_n_cats:]:
#                 print(' 置空低频值: ' + str(val))
#                 df_dum.loc[(df_dum[col]==val), col] = np.NaN
#             # 生成one-hot编码列
#             df_dum = pd.get_dummies(df_dum,dummy_na=True,columns=[col])
#             # 把全部是空值的行的众数dummy列置1
#             df_dum.loc[(df_dum[(col + '_nan')]==1), (col + '_' + str(mode))] = 1
        else:
            # 生成one-hot编码列
            #  print(col+' is in df_dum.columns?'+str(col in df_dum.columns))
            # print(df_dum.columns.to_list())
            df = pd.get_dummies(df, columns=[col])
    return df

# 打印一个df的na值
def print_na(df):
    # 检查d12 nan值
    df_na = df.isna().sum()
    for idx in df_na.index.to_list():
        if (df_na[idx]!=0):
            print(str(idx)+' '+str(df_na[idx]))


In [4]:
### intersection
# ''' d01&d02 '''
# id_train = set(d01['申请编号'])
# len(id_train)
# id_test = set(d02['申请编号'])
# len(id_test)
# inter = id_train.intersection(id_test)
# len(inter)

In [5]:
### 过滤

# 删除特征'是否为最后一次申请记录'
if ('是否为最后一次申请记录' in d3.columns):
    d3.drop(columns='是否为最后一次申请记录',inplace=True)

# 过滤合同状态为'拒绝'的样本
d3 = d3[d3['合同状态']!=2]

In [6]:
### Merge d1 d2 ###
uid = '申请编号'
d12 = pd.merge(d1, d2, how='inner', on=uid)
d12.shape

(161511, 62)

In [7]:
### d12 异常值处理

d12.loc[
    (d12['孩子个数'] == -1), '孩子个数'] = get_mode(d12, '孩子个数')

# 距申请日期>0, 置平均值
d12_cols = {}
d12_cols['_dt'] = [
    '最近一次换手机号码距申请日天数',
    '出生日期距申请日期天数',
    '工作日期距申请日期天数',
    '注册日期距申请日期天数',
    '身份认证日期距申请日期天数']

for col in d12_cols['_dt']:
    mean = d12[col].mean()
    d12.loc[d12[col] > 0, col] = mean

### Outlier：Get ###
low = .01
high = .99
quant_df = d12.drop([uid], axis=1).quantile([low, high]).T
quant_df.to_csv('./tmp/1_d12_outlier.csv')

### Outlier：Apply ###

outliers = pd.read_csv('./tmp/0_outlier1.csv', index_col=0)
outliers.shape
outliers.head()

def set_outlier(col):
    if col < col_min:
        col = col_min
    elif col > col_max:
        col = col_max
    return col

# def set_outlier(col):
#     if col < col_min or col > col_max:
#         col = np.NaN
#     return col
    
for col in outliers.index:
# for col in '出生日期距申请日期天数', '社交圈违约信息1_2':
    col_min = outliers.loc[col, 'min']
    col_max = outliers.loc[col, 'max']
    d12[col] = d12[col].apply(set_outlier)
    
# d12.to_csv(f'./tmp/2_eda_outlier.csv')

(61, 2)

,min,max
地址是否一致标志1,0,1
地址是否一致标志2,0,1
地址是否一致标志3,0,1
地址是否一致标志4,0,1
地址是否一致标志5,0,1


In [8]:
# 检查d12 nan值
'''d12 nan值'''
print_na(d12)

# d12['收入类型'].value_counts(dropna=False)
# d12_dum.columns.to_list()

'd12 nan值'

贷款年金 8
商品价格 140
陪同申请人 675
外部评分 90682
贷款申请前1小时内征信查询次数 22217
贷款申请前1天内征信查询次数 22217
贷款申请前1周内征信查询次数 22217
贷款申请前1个月内征信查询次数 22217
贷款申请前1个季度内征信查询次数 22217
贷款申请前1年内征信查询次数 22217
有车时间 106448
职业 48412
家庭成员数 1
居住信息1 107830
居住信息2 113207
居住信息3 81643
居住信息4 78587
社交圈违约信息2_2 516
社交圈违约信息2_1 516
社交圈违约信息1_2 516
社交圈违约信息1_1 516


In [9]:
### d12 数据空值处理 ###

# 插补平均值
nan_to_mean = [
    '贷款年金',
    '商品价格',
    '外部评分',
    '贷款申请前1小时内征信查询次数',
    '贷款申请前1天内征信查询次数',
    '贷款申请前1周内征信查询次数',
    '贷款申请前1个月内征信查询次数',
    '贷款申请前1个季度内征信查询次数',
    '贷款申请前1年内征信查询次数',
    '居住信息1',
    '居住信息2',
    '居住信息3',
    '居住信息4',
    '有车时间'
]
d12 = fill_mean(d12, nan_to_mean)

nan_to_mode = [
    '家庭成员数',
    '社交圈违约信息2_2',
    '社交圈违约信息2_1',
    '社交圈违约信息1_2',
    '社交圈违约信息1_1',
    '陪同申请人',
    '职业'
]
d12 = fill_mode(d12, nan_to_mode)

# 检查d12 nan值
'''d12 nan值'''
print_na(d12)

'd12 nan值'

In [10]:
### d12 类型变量转dummy列 ###
cat_to_dummy = [
    '陪同申请人',
    '收入类型',
    '教育程度',
    '居住状态',
    '职业',
    '单位类型'
]

d12_dum = d12.copy()
d12_dum = dummy_catgorical(d12_dum, cat_to_dummy, 20)

d12.shape
d12_dum.shape
'''d12_dum nan值'''
print_na(d12_dum)

initial num of cols: 62
陪同申请人has7
收入类型has8
教育程度has5
居住状态has6
职业has18
单位类型has58
 col含有过多值, skipping


(161511, 62)

(161511, 101)

'd12_dum nan值'

In [11]:
### d12_dum 处理col含有过多值

if ('单位类型' in d12_dum.columns):
    d12_dum.drop(columns='单位类型',inplace=True)

In [12]:
# 检查d3 nan值
'''d3 nan值'''
print_na(d3)

'd3 nan值'

贷款类型 161
贷款年金 144172
首付金额 303668
商品价格 152099
首付率 303668
利率1 609789
利率2 609789
陪同申请人 311224
贷款期数 144171
产品组合 161
首付时间 166778


In [13]:
### d3数据清洗 - 金额型 ###
# d3_cols['curr'] = [
#     '贷款年金',
#     '申请额度',
#     '信用额度',
#     '首付金额',
#     '商品价格']

### d3数据清洗 - 时长型 ###
# d3_cols['time_span'] = [
#     '历史贷款授信距本次申请时间',
#     '首付时间']

# 异常值置NaN
d3.loc[
    (d3['首付时间'] == 292204.0), '首付时间'] = np.NaN

# d3['销售区域_count'] = d3.groupby('销售区域')['销售区域'].transform('count')
# d3.loc[d3['销售区域_count'] < 5000, '销售区域'] = np.NaN

### d3数据清洗 - 概率型 ###
# d3_cols['prob'] = [
#     '首付率',
#     '利率1',
#     '利率2']

# 插补平均值
nan_to_mean = [
    '贷款年金',
    '首付金额',
    '商品价格',
    '首付时间',
    '首付率',
    '利率1',
    '利率1',
    '利率2'
]
d3 = fill_mean(d3, nan_to_mean)

### d3数据清洗 - 布尔型 ###
# d3_cols['bool'] = [
#     '是否为当天最后一次申请记录']

### d3数据清洗 - 计数型 ###
# d3_cols['cnt'] = [
#     '贷款期数']

# 插补众数
nan_to_mode = [
    '贷款期数',
    '贷款类型',
    '陪同申请人',
    '产品组合'
]
d3 = fill_mode(d3, nan_to_mode)

d3.shape
'''d3 nan值'''
print_na(d3)

(612366, 31)

'd3 nan值'

In [14]:
### d3 类型变量转dummy列 ###
# d3_dum = d3.copy()

# d3_dum.loc[(d3_dum['贷款用途'] != 23) & (d3_dum['贷款用途'] != 24), '贷款用途'] = np.NaN

# d3_dum.loc[
#     (d3_dum['申请时点'] != 3) & 
#     (d3_dum['申请时点'] != 4) &
#     (d3_dum['申请时点'] != 5) &
#     (d3_dum['申请时点'] != 6) &
#     (d3_dum['申请时点'] != 7) &
#     (d3_dum['申请时点'] != 8) &
#     (d3_dum['申请时点'] != 9), '申请时点'] = np.NaN


cat_to_dummy = [
    '贷款类型',
    '贷款用途',
    '合同状态',
    '付款方式',
    '拒绝原因',
    '陪同申请人',
    '客户类型',
    '商品类别',
    '组合类型',
    '产品类型',
    '获客渠道',
    '销售区域',
    '行业',
    '收益级别',
    '产品组合',
    '申请时点',
    '申请周内日'
]

d3_dum = d3.copy()
d3_dum = dummy_catgorical(d3, cat_to_dummy, 27)

d3_dum.shape
'''d3_dum nan值'''
print_na(d3_dum)

initial num of cols: 31
贷款类型has3
贷款用途has25
合同状态has3
付款方式has4
拒绝原因has3
陪同申请人has7
客户类型has4
商品类别has27
组合类型has5
产品类型has3
获客渠道has8
销售区域has1653
 col含有过多值, skipping
行业has11
收益级别has5
产品组合has17
申请时点has12
申请周内日has7


(612366, 159)

'd3_dum nan值'

In [15]:
### d3_dum 处理col含有过多值

if ('销售区域' in d3_dum.columns):
    d3_dum.drop(columns='销售区域',inplace=True)

In [20]:
### d3 group by ###

agg_dict = {}
d3_cols = {}
# 金额型
d3_cols['curr'] = [
    '贷款年金',
    '申请额度',
    '信用额度',
    '首付金额',
    '商品价格']

# 类别型
d3_cols['cat'] = [
    '贷款类型',
    '贷款用途',
    '合同状态',
    '付款方式',
    '拒绝原因',
    '陪同申请人',
    '客户类型',
    '商品类别',
    '组合类型',
    '产品类型',
    '获客渠道',
#     '销售区域',
    '行业',
    '收益级别',
    '产品组合']

# 时点型
d3_cols['time_point'] = [
    '申请周内日',
    '申请时点']

# 时长型
d3_cols['time_span'] = [
    '历史贷款授信距本次申请时间',
    '首付时间']

# 布尔型
d3_cols['bool'] = [
#     '是否为最后一次申请记录',
    '是否为当天最后一次申请记录']

# 概率型
d3_cols['prob'] = [
    '首付率',
    '利率1',
    '利率2']

# 计数型
d3_cols['cnt'] = [
    '贷款期数']

# 数值型 - 金额
curr_agg = ['mean', 'max', 'min']
for col in d3_cols['curr']:
    agg_dict[col] = curr_agg

# 数值型 - 时长 
time_span_agg = ['mean', 'max', 'min']
for col in d3_cols['time_span']:
    agg_dict[col] = time_span_agg

# 数值型 - 概率
prob_agg = ['mean', 'max', 'min']
for col in d3_cols['prob']:
    agg_dict[col] = prob_agg

# 离散型 - 类型
cat_agg = ['sum', 'mean']
for col1 in d3_dum.columns:
    for col2 in d3_cols['cat']:
        if (col2 in col1):
            agg_dict[col1] = cat_agg

# 离散型 - 时点
time_point_agg = ['sum', 'mean']
for col1 in d3_dum.columns:
    for col2 in d3_cols['time_point']:
        if (col2 in col1):
            agg_dict[col1] = time_point_agg


# print(agg_dict)

d3_dum = d3_dum.groupby('申请编号').agg(agg_dict)

d3_dum.columns = pd.Series(d3_dum.columns.tolist()).apply(pd.Series).sum(axis=1)

d3_dum.shape

(152768, 318)

In [27]:
# 检查d4 nan值
'''d4 nan值'''
print_na(d4)

'd4 nan值'

In [26]:
# for col in d4.columns:
#     d4[col].value_counts()

# 距申请日期>0, 置平均值
# d4_cols = {}
# d4_cols['_dt'] = [
#     '分期付款实际支付时间']
    
nan_to_mean = [
    '分期付款实际支付时间',
    '本期还款金额'
]
d4 = fill_mean(d4,nan_to_mean)


In [31]:
### d3 group by ###

if ('分期付款日期' in d4.columns):
    d4.drop(columns='分期付款日期',inplace=True)
    
d4.shape

agg_dict = {}
d4_cols = {}
# 金额型
d4_cols['curr'] = [
    '本期贷款金额',
    '本期还款金额']


# 时长型
d4_cols['time_span'] = [
    '分期付款应支付时间',
    '分期付款实际支付时间']

# 计数型
d4_cols['cnt'] = [
    '有支付行为的分期付款']

# 数值型 - 金额
curr_agg = ['mean', 'max', 'min']
for col in d4_cols['curr']:
    agg_dict[col] = curr_agg

# 数值型 - 时长 
time_span_agg = ['mean', 'max', 'min']
for col in d4_cols['time_span']:
    agg_dict[col] = time_span_agg


# print(agg_dict)

d4 = d4.groupby('申请编号').agg(agg_dict)

d4.columns = pd.Series(d4.columns.tolist()).apply(pd.Series).sum(axis=1)

d4.shape


(6058093, 7)

(153309, 12)

In [32]:
### Merge d12_dum and d3_dum ###
d12_d3_dum = pd.merge(d12_dum, d3_dum, how='left', on='申请编号')
d12_d3_dum.shape



(161511, 418)

In [33]:
### Merge d12_d3_dum and d4 ###
d1234 = pd.merge(d12_d3_dum, d4, how='left', on='申请编号')
d1234.shape

(161511, 430)

In [35]:
# print_na(d1234)

In [36]:
# Fill Null

imp =Imputer(missing_values=np.NaN, strategy="mean", axis=0)
data = pd.DataFrame(imp.fit_transform(d1234))
data.columns = d1234.columns
data.shape
data.head()

/usr/local/bin/anaconda3/envs/python37/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


(161511, 430)

,申请编号,贷款类型,信用额度,贷款年金,商品价格,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,身份认证日期距申请日期天数,是否提供手机号,...,本期贷款金额min,本期还款金额mean,本期还款金额max,本期还款金额min,分期付款应支付时间mean,分期付款应支付时间max,分期付款应支付时间min,分期付款实际支付时间mean,分期付款实际支付时间max,分期付款实际支付时间min
0,0.0,0.0,460190.889355,17463.042019,419951.511045,-10107.0,-342.0,-5421.0,-3292.0,1.0,...,1072.0,880.266667,1100.0,12.0,-1600.000000,-1480.0,-1744.0,-1610.333333,-1490.0,-1758.0
1,1.0,0.0,424370.659603,15585.046388,384597.975692,-13980.0,-1110.0,-3387.0,-826.0,1.0,...,635.0,778.520000,976.0,242.0,-267.920000,-19.0,-518.0,-272.480000,-23.0,-527.0
2,2.0,0.0,469330.587153,18837.558252,419951.511045,-13331.0,-2246.0,-3870.0,-171.0,1.0,...,2574.0,5455.414634,39611.0,607.0,-331.829268,-4.0,-641.0,-334.439024,-28.0,-648.0
3,3.0,0.0,464188.353140,16754.587069,409850.500944,-16540.0,0.0,-970.0,-2916.0,0.0,...,4906.0,4911.500000,4912.0,4906.0,-252.000000,-120.0,-384.0,-267.000000,-134.0,-396.0
4,5.0,0.0,434196.988738,16165.009200,394698.985793,-17919.0,-11037.0,-9350.0,-3588.0,1.0,...,33.0,1923.222222,17350.0,33.0,-1107.925926,-222.0,-2306.0,-1111.407407,-222.0,-2306.0


In [ ]:
data['申请编号'] = data['申请编号'].apply(round)

prefix = 'train'
data_train = pd.merge(d01['申请编号'], data, on='申请编号', how='left')
data_train.shape
data_train.to_csv(f'./tmp/{prefix}_d1234.csv')

prefix = 'A'
data_test = pd.merge(d05['申请编号'], data, on='申请编号', how='left')
data_test.shape
data_test.to_csv(f'./tmp/{prefix}_d1234.csv')

(140000, 430)

In [ ]:
for col in data_train.columns:
    print(col)